## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-15-15-57-00 +0000,wsj,Goldman Sachs Profit Rises on Jump in Dealmaki...,https://www.wsj.com/finance/banking/goldman-sa...
1,2026-01-15-15-57-00 +0000,wsj,President Trump threatened to invoke the Insur...,https://www.wsj.com/politics/policy/trump-minn...
2,2026-01-15-15-56-52 +0000,nyt,"Under Trump, a Shift Toward ‘Absolute Immunity...",https://www.nytimes.com/2026/01/15/us/politics...
3,2026-01-15-15-55-45 +0000,nypost,Alleged killer husband Brendan Banfield put na...,https://nypost.com/2026/01/15/us-news/alleged-...
4,2026-01-15-15-55-16 +0000,nyt,What Are Trump’s Options in Iran?,https://www.nytimes.com/video/world/middleeast...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2368/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
8,trump,69
49,iran,34
29,ice,22
76,new,19
197,venezuela,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
263,2026-01-14-22-56-00 +0000,wsj,Top officials from Denmark and Greenland said ...,https://www.wsj.com/politics/national-security...,145
329,2026-01-14-20-17-00 +0000,wsj,"Fed governor Stephen Miran, an appointee of Pr...",https://www.wsj.com/economy/central-banking/fe...,145
238,2026-01-15-00-54-00 +0000,wsj,The Senate voted to kill a war-powers resoluti...,https://www.wsj.com/politics/policy/trump-beat...,141
1,2026-01-15-15-57-00 +0000,wsj,President Trump threatened to invoke the Insur...,https://www.wsj.com/politics/policy/trump-minn...,138
79,2026-01-15-13-36-16 +0000,nypost,Trump threatens to invoke Insurrection Act aga...,https://nypost.com/2026/01/15/us-news/trump-th...,137


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
263,145,2026-01-14-22-56-00 +0000,wsj,Top officials from Denmark and Greenland said ...,https://www.wsj.com/politics/national-security...
216,69,2026-01-15-02-13-51 +0000,cbc,Video shows woman dragged from car by ICE agen...,https://www.cbc.ca/news/world/ice-woman-car-mi...
5,67,2026-01-15-15-55-11 +0000,nypost,U-Haul driver arrested after plowing into Iran...,https://nypost.com/2026/01/15/us-news/u-haul-d...
238,65,2026-01-15-00-54-00 +0000,wsj,The Senate voted to kill a war-powers resoluti...,https://www.wsj.com/politics/policy/trump-beat...
329,64,2026-01-14-20-17-00 +0000,wsj,"Fed governor Stephen Miran, an appointee of Pr...",https://www.wsj.com/economy/central-banking/fe...
315,61,2026-01-14-20-50-09 +0000,bbc,US launches phase two of Gaza peace plan with ...,https://www.bbc.com/news/articles/cjw1nxe5pvlo...
1,60,2026-01-15-15-57-00 +0000,wsj,President Trump threatened to invoke the Insur...,https://www.wsj.com/politics/policy/trump-minn...
35,46,2026-01-15-15-06-43 +0000,nypost,2 dead in Thailand after crane crashes onto ro...,https://nypost.com/2026/01/15/world-news/two-d...
360,41,2026-01-14-18-27-54 +0000,latimes,Child porn concerns prompt California to launc...,https://www.latimes.com/california/story/2026-...
102,38,2026-01-15-12-16-30 +0000,nypost,Mamdani’s radical tenant advocate Cea Weaver o...,https://nypost.com/2026/01/15/us-news/mayor-ma...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
